In [1]:
from multiprocessing import Process, Queue
import subprocess

from run_info_pane import run_info_pane
from run_node_app import run_node_app

def run_xpra_window(queue):
    script_target = 'run_two_window_app.py'
    x11_screen = 80
    xpra_target_port = '8080'
    user = 'myuser'
    
    current_working_directory = subprocess.check_output('pwd')
    current_working_directory = current_working_directory.decode().strip()
    script_filepath = f'{current_working_directory}/{script_target}'

    base_command = f'xpra start :{x11_screen}'
    flags = [
        f'--bind-tcp=0.0.0.0:{xpra_target_port}',
        '--mdns=no',
        '--webcam=no',
        '--no-daemon',
        '--pulseaudio=no',
        f'--start="python3 {script_filepath}"'
    ]
    
    flags = ' '.join(flags)
     
    command = f"su myuser -c '{base_command} {flags}'"
    print(command)
    subprocess.call(command, shell=True)
    

In [ ]:
# run_xpra_window(None)
!su myuser -c 'xpra start :80 --bind-tcp=0.0.0.0:8080 --mdns=no --webcam=no --no-daemon --pulseaudio=no --start="python3 /srv/node_app/run_two_window_app.py"'

2023-02-07 03:21:27,709 Warning: failed to create script directory '/run/user/1000/xpra':
2023-02-07 03:21:27,709  [Errno 2] No such file or directory: '/run/user/1000/xpra'
2023-02-07 03:21:27,710  ($XDG_RUNTIME_DIR has not been created?)
2023-02-07 03:21:27,847 created tcp socket '0.0.0.0:8080'
2023-02-07 03:21:27,849 cannot access python uinput module:
2023-02-07 03:21:27,849  No module named 'uinput'
_XSERVTransmkdir: Mode of /tmp/.X11-unix should be set to 1777
_XSERVTransSocketCreateListener: failed to bind listener
_XSERVTransSocketUNIXCreateListener: ...SocketCreateListener() failed
_XSERVTransMakeAllCOTSServerListeners: failed to create listener for unix
2023-02-07 03:21:27,875 dbus-launch failed to start using command 'dbus-launch --sh-syntax --close-stderr':

2023-02-07 03:21:27,876  exit code is 255

2023-02-07 03:21:28,256 Warning: failed to create socket directory '/run/user/1000/xpra'
2023-02-07 03:21:28,256  [Errno 2] No such file or directory: '/run/user/1000/xpra'
202

127.0.0.1 - - [07/Feb/2023 03:21:40] "POST /socket.io/?EIO=4&transport=polling&t=OOfvBMx&sid=3-NSbBBMsWIn7yhbAAAE HTTP/1.1" 200 -
127.0.0.1 - - [07/Feb/2023 03:21:40] "GET /socket.io/?EIO=4&transport=polling&t=OOfvBMy&sid=3-NSbBBMsWIn7yhbAAAE HTTP/1.1" 200 -
2023-02-07 03:21:41,001 Handshake complete; enabling connection
2023-02-07 03:21:41,044 dbus server error
Traceback (most recent call last):
  File "/usr/lib/python3/dist-packages/xpra/server/dbus/dbus_common.py", line 12, in dbus_exception_wrap
    v = fn()
  File "/usr/lib/python3/dist-packages/xpra/server/source/dbus_mixin.py", line 29, in make_dbus_server
    return DBUS_Source(self, os.environ.get("DISPLAY", "").lstrip(":"))
  File "/usr/lib/python3/dist-packages/xpra/server/dbus/dbus_source.py", line 37, in __init__
    session_bus = init_session_bus()
  File "/usr/lib/python3/dist-packages/xpra/dbus/common.py", line 22, in init_session_bus
    _session_bus = dbus.SessionBus(private=private)
  File "/usr/lib/python3/dist-pack

In [2]:
if __name__ == '__main__':
    queue = Queue()
    xpra_process = Process(target=run_node_app, args=(queue,))
    info_pane_process = Process(target=run_info_pane, args=(queue,))
    xpra_process.start()
    info_pane_process.start()

2023-02-07 02:16:48.538230
0:00:00.010617


WebSocket transport not available. Install simple-websocket for improved performance.
Werkzeug appears to be used in a production deployment. Consider switching to a production web server instead.


 * Serving Flask app 'run_info_pane'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
QStandardPaths: XDG_RUNTIME_DIR not set, defaulting to '/tmp/runtime-root'
The WebSocket transport is not available, you must install a WebSocket server that is compatible with your async mode to enable it. See the documentation for details. (further occurrences of this error will be logged with level INFO)
127.0.0.1 - - [07/Feb/2023 02:16:54] "GET /socket.io/?EIO=4&transport=polling&t=OOfgMal HTTP/1.1" 200 -
127.0.0.1 - - [07/Feb/2023 02:16:54] "POST /socket.io/?EIO=4&transport=polling&t=OOfgMg1&sid=UGQHEADMZcmH3exWAAAA HTTP/1.1" 200 -
127.0.0.1 - - [07/Feb/2023 02:16:54] "GET /socket.io/?EIO=4&transport=polling&t=OOfgMg3&sid=UGQHEADMZcmH3exWAAAA HTTP/1.1" 200 -
127.0.0.1 - - [07/Feb/2023 02:16:54] "POST /socket.io/?EIO=4&transport=polling&t=OOfgMlD&sid=UGQHEADMZcmH3exWAAAA HTTP/1.1" 200 -
127.0.0.1 - - [07/Feb/2023 02:16:54] "GET /socket.io/?EIO=4&transport=polling&t=OOfgMlE&sid=UGQHEADMZcmH3exWAAAA HTTP/1.1" 200 -
127.0.0.1 - 